In [1]:
import tensorflow as tf
name_file='https://drive.usercontent.google.com/download?id=1DoB_MJHXDRaKvjAwHzPNZHdmx0Gl87nd&export=download&authuser=0&confirm=t&uuid=798deac7-ba99-4116-bb8b-7cb3ef043ce8&at=APZUnTUg8mVcxXbOXkJx3qS3K9cu:1723001283357'
import urllib.request
urllib.request.urlretrieve(
        name_file, 'model_tensor_googledrive.hdf5')
print("url berhasil")
model = tf.keras.models.load_model('model_tensor_googledrive.hdf5')
print(model.summary())

url berhasil
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 240, 3)            0         
                                                                 
 conv1d (Conv1D)             (None, 240, 120)          10920     
                                                                 
 batch_normalization (BatchN  (None, 240, 120)         480       
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 240, 120)          864120    
                                                                 
 dropout (Dropout)           (None, 240, 120)          0         
                                                                 
 batch_normalization_1 (Batc  (None, 240, 120)         480       
 hNormalization)                           

In [3]:
import pandas as pd
train_name_cells=['S1/B2/C1']
test_name_cells=['S7/B2/C1']
file_path="D:\\KULIAH\\teep\\AI\\dataset\\08_01_2024\\CellReports.csv"
df_begin=pd.read_csv(file_path)
df_1=df_begin.copy()
convert_time=pd.to_datetime(df_1['timestamp'], unit='ms',origin='unix')
df_1.insert(df_1.columns.get_loc('timestamp') + 1, 'datetime_column', convert_time)
df_1.insert(df_1.columns.get_loc('datetime_column') + 2, 'hour', df_1['datetime_column'].dt.hour+df_1['datetime_column'].dt.minute/60)
df_1.set_index('datetime_column', inplace=True)
df_1.drop(columns=['timestamp'], inplace=True)

# Find columns where correlation with 'RRU.PrbUsedDl'is greater than 0.5
columns_with_high_corr = ['RRU.PrbUsedDl','RRC.ConnMean', 'DRB.UEThpDl']
seleted_columns = ['Viavi.Cell.Name']+ columns_with_high_corr 
df_2= df_1[seleted_columns].copy()
#cell_name= test_name_cells+train_name_cells
df= df_2[df_2['Viavi.Cell.Name'].isin(train_name_cells)].copy()
df= df.drop(columns=['Viavi.Cell.Name']).astype('int').copy()
df

,RRU.PrbUsedDl,RRC.ConnMean,DRB.UEThpDl
datetime_column,,,
2024-08-01 00:00:00,35,1,0
2024-08-01 00:01:00,20,1,0
2024-08-01 00:02:00,30,1,0
2024-08-01 00:03:00,34,1,0
2024-08-01 00:04:00,25,1,0
...,...,...,...
2024-08-08 23:55:00,25,1,0
2024-08-08 23:56:00,24,1,0
2024-08-08 23:57:00,59,2,0


In [5]:
input_width = 60*4
label_width = 60
numpy_test= df.tail(input_width).to_numpy().reshape(-1,input_width,3)
numpy_test= numpy_test.round(2)
print(numpy_test.shape)
numpy_test.tolist()

(1, 240, 3)


[[[59, 2, 0],
  [64, 2, 0],
  [60, 3, 0],
  [78, 2, 0],
  [51, 1, 0],
  [37, 1, 0],
  [11, 0, 0],
  [53, 3, 0],
  [45, 2, 0],
  [48, 2, 0],
  [51, 1, 0],
  [63, 2, 0],
  [61, 3, 0],
  [42, 3, 0],
  [76, 3, 0],
  [79, 3, 0],
  [67, 4, 0],
  [78, 3, 0],
  [88, 3, 0],
  [92, 4, 0],
  [77, 3, 0],
  [61, 4, 0],
  [83, 5, 0],
  [51, 2, 0],
  [73, 3, 0],
  [46, 2, 0],
  [75, 3, 0],
  [25, 0, 0],
  [56, 1, 0],
  [70, 2, 0],
  [40, 2, 0],
  [36, 2, 0],
  [28, 1, 0],
  [42, 1, 0],
  [68, 3, 0],
  [58, 2, 0],
  [55, 1, 0],
  [83, 3, 0],
  [68, 1, 0],
  [66, 2, 0],
  [79, 2, 0],
  [45, 1, 0],
  [55, 2, 0],
  [50, 3, 0],
  [43, 3, 0],
  [82, 4, 0],
  [2, 0, 0],
  [33, 1, 0],
  [11, 1, 0],
  [49, 3, 0],
  [39, 1, 0],
  [71, 3, 0],
  [73, 2, 0],
  [66, 2, 0],
  [66, 3, 0],
  [58, 2, 0],
  [64, 2, 0],
  [37, 1, 0],
  [100, 2, 0],
  [100, 2, 0],
  [35, 1, 0],
  [36, 1, 0],
  [13, 0, 0],
  [55, 1, 0],
  [64, 2, 0],
  [42, 1, 0],
  [28, 0, 0],
  [39, 0, 0],
  [51, 2, 0],
  [39, 1, 0],
  [57, 3, 0],
  [36

In [11]:
# Create a dictionary to hold the data
import json
data = {
    "signature_name": "serving_default",
    'instances': numpy_test.tolist()
}

# Save the dictionary to a JSON file
with open('input_qoe.json', 'w') as json_file:
    json.dump(data, json_file)

print("Array saved to data.json")

Array saved to data.json


In [12]:
import json
with open('input_qoe.json', 'r') as json_file: data = json.load(json_file)
print(data)

{'signature_name': 'serving_default', 'instances': [[[59, 2, 0], [64, 2, 0], [60, 3, 0], [78, 2, 0], [51, 1, 0], [37, 1, 0], [11, 0, 0], [53, 3, 0], [45, 2, 0], [48, 2, 0], [51, 1, 0], [63, 2, 0], [61, 3, 0], [42, 3, 0], [76, 3, 0], [79, 3, 0], [67, 4, 0], [78, 3, 0], [88, 3, 0], [92, 4, 0], [77, 3, 0], [61, 4, 0], [83, 5, 0], [51, 2, 0], [73, 3, 0], [46, 2, 0], [75, 3, 0], [25, 0, 0], [56, 1, 0], [70, 2, 0], [40, 2, 0], [36, 2, 0], [28, 1, 0], [42, 1, 0], [68, 3, 0], [58, 2, 0], [55, 1, 0], [83, 3, 0], [68, 1, 0], [66, 2, 0], [79, 2, 0], [45, 1, 0], [55, 2, 0], [50, 3, 0], [43, 3, 0], [82, 4, 0], [2, 0, 0], [33, 1, 0], [11, 1, 0], [49, 3, 0], [39, 1, 0], [71, 3, 0], [73, 2, 0], [66, 2, 0], [66, 3, 0], [58, 2, 0], [64, 2, 0], [37, 1, 0], [100, 2, 0], [100, 2, 0], [35, 1, 0], [36, 1, 0], [13, 0, 0], [55, 1, 0], [64, 2, 0], [42, 1, 0], [28, 0, 0], [39, 0, 0], [51, 2, 0], [39, 1, 0], [57, 3, 0], [36, 3, 0], [43, 2, 0], [53, 2, 0], [77, 3, 0], [55, 2, 0], [54, 2, 0], [42, 2, 0], [43, 1, 0]

In [16]:
import numpy as np
np.shape(data['instances'])

(1, 240, 3)

In [17]:
data['instances']

[[[59, 2, 0],
  [64, 2, 0],
  [60, 3, 0],
  [78, 2, 0],
  [51, 1, 0],
  [37, 1, 0],
  [11, 0, 0],
  [53, 3, 0],
  [45, 2, 0],
  [48, 2, 0],
  [51, 1, 0],
  [63, 2, 0],
  [61, 3, 0],
  [42, 3, 0],
  [76, 3, 0],
  [79, 3, 0],
  [67, 4, 0],
  [78, 3, 0],
  [88, 3, 0],
  [92, 4, 0],
  [77, 3, 0],
  [61, 4, 0],
  [83, 5, 0],
  [51, 2, 0],
  [73, 3, 0],
  [46, 2, 0],
  [75, 3, 0],
  [25, 0, 0],
  [56, 1, 0],
  [70, 2, 0],
  [40, 2, 0],
  [36, 2, 0],
  [28, 1, 0],
  [42, 1, 0],
  [68, 3, 0],
  [58, 2, 0],
  [55, 1, 0],
  [83, 3, 0],
  [68, 1, 0],
  [66, 2, 0],
  [79, 2, 0],
  [45, 1, 0],
  [55, 2, 0],
  [50, 3, 0],
  [43, 3, 0],
  [82, 4, 0],
  [2, 0, 0],
  [33, 1, 0],
  [11, 1, 0],
  [49, 3, 0],
  [39, 1, 0],
  [71, 3, 0],
  [73, 2, 0],
  [66, 2, 0],
  [66, 3, 0],
  [58, 2, 0],
  [64, 2, 0],
  [37, 1, 0],
  [100, 2, 0],
  [100, 2, 0],
  [35, 1, 0],
  [36, 1, 0],
  [13, 0, 0],
  [55, 1, 0],
  [64, 2, 0],
  [42, 1, 0],
  [28, 0, 0],
  [39, 0, 0],
  [51, 2, 0],
  [39, 1, 0],
  [57, 3, 0],
  [36

In [18]:
model.predict(data['instances'])

1/1 [==============================] - 0s 383ms/step


array([[[ 31.016045  ],
        [ 33.98409   ],
        [ 16.397655  ],
        [ 34.127796  ],
        [ 20.433165  ],
        [ 21.692772  ],
        [ 13.858817  ],
        [ 27.967295  ],
        [ 18.230024  ],
        [ 23.515512  ],
        [ 14.003301  ],
        [ 22.6701    ],
        [ 16.657562  ],
        [ 20.312626  ],
        [ 24.819527  ],
        [  1.9386809 ],
        [ 24.11949   ],
        [ 17.447315  ],
        [ 15.206126  ],
        [ 10.95967   ],
        [  6.7770047 ],
        [ 11.822324  ],
        [  9.981428  ],
        [  1.0420895 ],
        [ 17.123814  ],
        [ 13.656023  ],
        [ 18.519842  ],
        [  2.494936  ],
        [ 12.732058  ],
        [ 13.787475  ],
        [ 11.663088  ],
        [ 10.746876  ],
        [ 10.15865   ],
        [  7.5434184 ],
        [  7.2578535 ],
        [  4.7520943 ],
        [ 11.022757  ],
        [ -1.6538335 ],
        [  9.981262  ],
        [ -8.523576  ],
        [-16.324646  ],
        [  0.241